In [1]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [2]:
# Quantum Phase Estimation (QPE) using Qiskit 2.x

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

def qpe_circuit(num_count_qubits, unitary, theta):
    """
    Constructs the Quantum Phase Estimation (QPE) circuit.
    num_count_qubits: number of counting qubits
    unitary: unitary gate (to apply controlled operations)
    theta: phase parameter (for U gate)
    """
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Apply Hadamard gates on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate (|1>) for the target qubit
    qc.x(num_count_qubits)

    # Step 3: Apply controlled unitary operations
    for qubit in range(num_count_qubits):
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    # Step 4: Apply inverse QFT to counting qubits
    inverse_qft(qc, num_count_qubits)

    # Step 5: Measure counting qubits
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    return qc

In [3]:
def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
        qc.h(j)
    return qc

In [4]:
def run_qpe(num_count_qubits=3, theta=0.125):
    """Executes the QPE circuit and visualizes the phase estimation result."""
    simulator = AerSimulator()
    qc = qpe_circuit(num_count_qubits, "U", theta)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print(qc.draw(output='text'))

In [5]:
if __name__ == "__main__":
    num_count_qubits = 3
    theta = 0.125  # phase value (1/8)
    print(f"Running Quantum Phase Estimation with {num_count_qubits} counting qubits and phase {theta}")
    run_qpe(num_count_qubits, theta)

Running Quantum Phase Estimation with 3 counting qubits and phase 0.125
     ┌───┐                            ┌───┐                                   »
q_0: ┤ H ├─■────────────────────────X─┤ H ├─■──────────────■──────────────────»
     ├───┤ │                        │ └───┘ │P(-π/2) ┌───┐ │                  »
q_1: ┤ H ├─┼────────■───────────────┼───────■────────┤ H ├─┼─────────■────────»
     ├───┤ │        │               │                └───┘ │P(-π/4)  │P(-π/2) »
q_2: ┤ H ├─┼────────┼────────■──────X──────────────────────■─────────■────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)                                            »
q_3: ┤ X ├─■────────■────────■────────────────────────────────────────────────»
     └───┘                                                                    »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«     ┌─┐           
«q_0: ┤M├───────────
«     

### Task 1

In [6]:
# Try different theta values
theta_values = [0.25, 0.375, 0.5]

for theta in theta_values:
    print(f"Running Quantum Phase Estimation with {num_count_qubits} counting qubits and phase {theta}")
    run_qpe(num_count_qubits, theta)

Running Quantum Phase Estimation with 3 counting qubits and phase 0.25
     ┌───┐                           ┌───┐                                   »
q_0: ┤ H ├─■───────────────────────X─┤ H ├─■──────────────■──────────────────»
     ├───┤ │                       │ └───┘ │P(-π/2) ┌───┐ │                  »
q_1: ┤ H ├─┼────────■──────────────┼───────■────────┤ H ├─┼─────────■────────»
     ├───┤ │        │              │                └───┘ │P(-π/4)  │P(-π/2) »
q_2: ┤ H ├─┼────────┼──────■───────X──────────────────────■─────────■────────»
     ├───┤ │P(π/2)  │P(π)  │P(2π)                                            »
q_3: ┤ X ├─■────────■──────■─────────────────────────────────────────────────»
     └───┘                                                                   »
c: 3/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌─┐           
«q_0: ┤M├───────────
«     └╥┘     ┌─┐ 

### Task 2

In [7]:
# Increase the number of counting qubits
num_count_qubits_values = [4, 5]
theta = 0.125 # Use the original theta value

for num_count_qubits in num_count_qubits_values:
    print(f"Running Quantum Phase Estimation with {num_count_qubits} counting qubits and phase {theta}")
    run_qpe(num_count_qubits, theta)

Running Quantum Phase Estimation with 4 counting qubits and phase 0.125
     ┌───┐                                    ┌───┐                         »
q_0: ┤ H ├─■────────────────────────────────X─┤ H ├─■──────────────■────────»
     ├───┤ │                                │ └───┘ │P(-π/2) ┌───┐ │        »
q_1: ┤ H ├─┼────────■─────────────────X─────┼───────■────────┤ H ├─┼────────»
     ├───┤ │        │                 │     │                └───┘ │P(-π/4) »
q_2: ┤ H ├─┼────────┼────────■────────X─────┼──────────────────────■────────»
     ├───┤ │        │        │              │                               »
q_3: ┤ H ├─┼────────┼────────┼──────■───────X───────────────────────────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)  │P(2π)                                  »
q_4: ┤ X ├─■────────■────────■──────■───────────────────────────────────────»
     └───┘                                                                  »
c: 4/═════════════════════════════════════════════════════════════════

### Task 3

In [8]:
def theoretical_phase_binary(theta, num_count_qubits):
    """Calculates the theoretical binary representation of the phase."""
    # The theoretical output is theta * 2^num_count_qubits
    theoretical_value = theta * (2 ** num_count_qubits)
    # The integer part of this value in binary is the expected measurement
    binary_representation = bin(int(theoretical_value))[2:]
    # Pad with leading zeros if necessary
    padded_binary = binary_representation.zfill(num_count_qubits)
    return padded_binary

# Example usage with the last run parameters
print(f"Theoretical binary for theta={theta} with {num_count_qubits} qubits: {theoretical_phase_binary(theta, num_count_qubits)}")

# To compare with simulation results:
# Look at the histograms generated by run_qpe. The most frequent outcome (the peak of the histogram)
# should correspond to the theoretical binary representation calculated above.
# For example, with theta=0.125 and 3 qubits, the theoretical value is 0.125 * 2^3 = 1.0.
# The integer part is 1, which in 3-bit binary is 001.
# The histogram for this case should show the highest probability for the '001' measurement.

Theoretical binary for theta=0.125 with 5 qubits: 00100


### Task 4

In [10]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=77ab28c3eee4e432e974391dc18765b72e936c735d3e25491d86a28ecc079f0a
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [11]:
# Generate and visualize the QPE circuit before measurement
num_count_qubits = 3
theta = 0.125  # phase value (1/8)

qc_viz = qpe_circuit(num_count_qubits, "U", theta)

# Remove measurement for visualization purposes
qc_viz.remove_final_measurements()

# Visualize the circuit
qc_viz.draw('mpl')
plt.show()

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."

### Task 5

In [14]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Create a simple noise model
noise_model = NoiseModel()

# Add a depolarizing error to all single qubit gates
p_depol = 0.01  # Probability of depolarizing error
noise_model.add_all_qubit_quantum_error(depolarizing_error(p_depol, 1), ['h', 'x', 'rz', 'p', 'u1', 'u2', 'u3'])

# Add a depolarizing error to all two qubit gates
noise_model.add_all_qubit_quantum_error(depolarizing_error(p_depol, 2), ['cx', 'cz', 'cu1', 'cu3', 'cy', 'ch'])

print(noise_model)

NoiseModel:
  Basis gates: ['cu1', 'cu3', 'cx', 'cy', 'cz', 'h', 'id', 'p', 'rz', 'sx', 'u1', 'u2', 'u3', 'x']
  Instructions with noise: ['cu1', 'cy', 'x', 'cu3', 'rz', 'u2', 'u3', 'ch', 'h', 'cz', 'cx', 'u1', 'p']
  All-qubits errors: ['h', 'x', 'rz', 'p', 'u1', 'u2', 'u3', 'cx', 'cz', 'cu1', 'cu3', 'cy', 'ch']


In [13]:
# Run QPE with the noise model
def run_qpe_with_noise(num_count_qubits=3, theta=0.125, noise_model=None):
    """Executes the QPE circuit with a noise model and visualizes the phase estimation result."""
    simulator = AerSimulator(noise_model=noise_model)
    qc = qpe_circuit(num_count_qubits, "U", theta)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print(qc.draw(output='text'))

# Run QPE with noise
num_count_qubits = 3
theta = 0.125
print(f"Running Quantum Phase Estimation with {num_count_qubits} counting qubits, phase {theta}, and noise")
run_qpe_with_noise(num_count_qubits, theta, noise_model)

Running Quantum Phase Estimation with 3 counting qubits, phase 0.125, and noise
     ┌───┐                            ┌───┐                                   »
q_0: ┤ H ├─■────────────────────────X─┤ H ├─■──────────────■──────────────────»
     ├───┤ │                        │ └───┘ │P(-π/2) ┌───┐ │                  »
q_1: ┤ H ├─┼────────■───────────────┼───────■────────┤ H ├─┼─────────■────────»
     ├───┤ │        │               │                └───┘ │P(-π/4)  │P(-π/2) »
q_2: ┤ H ├─┼────────┼────────■──────X──────────────────────■─────────■────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)                                            »
q_3: ┤ X ├─■────────■────────■────────────────────────────────────────────────»
     └───┘                                                                    »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«     ┌─┐           
«q_0: ┤M├──────────